In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()

In [8]:
import dataiku

# client initialization:
client = dataiku.api_client().get_connection("postgresql")
conn.get_info()["params"]['password']

'dataiku'

In [6]:
conn = dataiku.api_client().get_connection("postgresql")
conn.get_location_info()["params"]["password"]

/home/centos/dataiku-dss-11.2.0/python/dataikuapi/dss/admin.py:66: DeprecationWarning: DSSConnection.get_location_info is deprecated, please use get_info
  warnings.warn("DSSConnection.get_location_info is deprecated, please use get_info", DeprecationWarning)


'dataiku'